In [ ]:
import pandas as pd
import numpy as np

In [ ]:
a = pd.read_csv("https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv")
a.sort_values(by=['denominazione_regione', 'data'], inplace=True)
regioni = a.denominazione_regione.unique()
nuovi_pos = [a[a.denominazione_regione == regione]["nuovi_positivi"].values.tolist() for regione in regioni]
k1 = pd.DataFrame(np.column_stack(nuovi_pos), columns=regioni)

In [ ]:
k1

In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

In [ ]:
import time
k1['date'] = pd.date_range(start='24/02/2020', periods=len(k1), freq='D')
k1.set_index('date')
k2=k1.filter(['Lazio'])
k2
#tf.timestamp(k2.date)

In [ ]:
train_size = int(len(k2) * 0.8)
test_size = len(k2) - train_size
train, test = k2.iloc[0:train_size], k2.iloc[train_size:len(k2)]
print(len(train), len(test))


In [ ]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [ ]:
time_steps = 7

# reshape to [samples, time_steps, n_features]

X_train, y_train = create_dataset(train, train.Lazio, time_steps)
X_test, y_test = create_dataset(test, test.Lazio, time_steps)

print(X_train.shape, y_train.shape)

In [ ]:
from tensorflow import keras
model = keras.Sequential()
model.add(keras.layers.LSTM(
  units=128,
  input_shape=(X_train.shape[1], X_train.shape[2])
))
model.add(keras.layers.Dense(units=1))
model.compile(
  loss='mean_squared_error',
  optimizer=keras.optimizers.Adam(0.001)
)


In [ ]:
xt=tf.convert_to_tensor(X_train, dtype=tf.float32)
yt=tf.convert_to_tensor(y_train, dtype=tf.float32)
xtt=tf.convert_to_tensor(X_test, dtype=tf.float32)
ytt=tf.convert_to_tensor(y_test, dtype=tf.float32)

In [ ]:
history = model.fit(
    xt, yt,
    epochs=30,
    batch_size=16,
    validation_split=0.1,
    verbose=1,
    shuffle=False
)

In [ ]:
y_pred = model.predict(xtt)


In [ ]:
print(y_pred)